# Tilt and Compliance Corrections for OBS Data: Continuous
### Xiaotao Yang @ Harvard University
This notebook contains examples of compliance corrections using local data on the disk. The functions for tilt and compliance corrections are in module seispy.obsmaster.

## Step 0. Load needed packages.
Some functions are imported from the utils.py and the obsmaster.py.

In [1]:
#import needed packages.
from seispy import utils
from seispy import obsmaster as obs
import sys
import time
import scipy
import obspy
import pyasdf
import datetime
import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
from obspy import UTCDateTime
from obspy.core import Stream,Trace
from IPython.display import clear_output
from obspy.clients.fdsn import Client

## Step 1. Download example data and save to disk
Set downloading and saving parameters. We download the same data as Figure 5 in Janiszewski et al. (2019).The following figure is from their paper for reference.
Earthquake information:
M 7.1 - Vanuatu
2012-02-02 13:34:40 (UTC)17.827°S 167.133°E 23.0 km depth

![title](embededfigs/JaniszewskiGJI2019Fig5.png)

We download all four components of the OBS data for the example stations.

In [2]:
"""
Set parameters for local data. This will be used by downloading and reading later.
"""
rawdatadir='../data/raw'

In [9]:
downloadexample=False #change to False to save run time if the data has already been downloaded.
if downloadexample:
    """
    Parameters for downloading data.
    """
    client=Client('IRIS')
    # get data from IRIS web service
    net="7D"
    stalist=["FN07A","G30A"]#["G03A","J35A","J44A","J65A"]
    starttime = obspy.UTCDateTime("2012_02_02_0_0_0")       
    endtime   = obspy.UTCDateTime("2012_02_03_0_0_0")
    
    syear = starttime.year
    smon = starttime.month
    sday = starttime.day
    shour = starttime.hour
    smin = starttime.minute
    ssec = starttime.second
    
    eyear = endtime.year
    emon = endtime.month
    eday = endtime.day
    ehour = endtime.hour
    emin = endtime.minute
    esec = endtime.second
    
    fnamebase = str(syear) + '_' + str(smon)+'_'+str(sday)+'_'+str(shour)+'_'+str(smin)+'_'+str(ssec)+\
            'T'+str(eyear) + '_' + str(emon)+'_'+str(eday)+'_'+str(ehour)+'_'+str(emin)+'_'+str(esec)+'.h5'
    fname = os.path.join(rawdatadir,fnamebase)
    rmresp=True #remove instrument response
    # parameters for butterworth filter
    samp_freq=10
    pfreqmin=0.002
    pfreqmax=samp_freq/2

    # prefilter information used when removing instrument responses
    f1 = 0.95*pfreqmin;f2=pfreqmin
    if 1.05*pfreqmax > 0.48*samp_freq:
        f3 = 0.45*samp_freq
        f4 = 0.48*samp_freq
    else:
        f3 = pfreqmax
        f4= 1.05*pfreqmax
    pre_filt  = [f1,f2,f3,f4]

    """
    Start downloading.
    """
    for ista in stalist:
        print('Downloading '+net+"."+ista+" ...")
        t0=time.time()
        """
        a. Download OBS data.
        """
        tr1,tr2,trZ,trP = obs.getdata(net,ista,starttime,endtime,samp_freq=samp_freq,
                                      plot=False,rmresp=rmresp,pre_filt=pre_filt)
        sta_inv=client.get_stations(network=net,station=ista,
                                    starttime=starttime,endtime=endtime,
                                    location='*',level='response')
        ta=time.time() - t0
        print('  downloaded '+net+"."+ista+" in "+str(ta)+" seconds.")
        """
        b. Save to ASDF file.
        """
#         year = trZ.stats.starttime.year
#         julday = trZ.stats.starttime.julday
#         hour = trZ.stats.starttime.hour
#         mnt = trZ.stats.starttime.minute
#         sec = trZ.stats.starttime.second
#         tstamp = str(year) + '.' + str(julday)+'T'+str(hour)+'-'+str(mnt)+'-'+str(sec)
#         fname = os.path.join(datadir,tstamp+'_L'+str(trZ.stats.endtime-trZ.stats.starttime)+'s.h5')
        
        tags=[]
        for itr,tr in enumerate([tr1,tr2,trZ,trP],1):
            if len(tr.stats.location) == 0:
                tlocation='00'
            else:
                tlocation=tr.stats.location

            tags.append(tr.stats.channel.lower()+'_'+tlocation.lower())

        print('  saving to '+fname)
        utils.save2asdf(fname,Stream(traces=[tr1,tr2,trZ,trP]),tags,sta_inv=sta_inv)


station 7D.FN07A --> pressure channel: HDH
station 7D.FN07A --> seismic channels: HH1, HH2, HHZ
  downsamping from 125 to 10
  removing response using inv for 7D.FN07A.HH1
  removing response using inv for 7D.FN07A.HH2
  removing response using inv for 7D.FN07A.HHZ
  removing response using inv for 7D.FN07A.HDH
  downloaded 7D.FN07A in 72.11439108848572 seconds.
  saving to 2012_2_2_0_0_0T2012_2_3_0_0_0.h5
station 7D.G30A --> pressure channel: BDH
station 7D.G30A --> seismic channels: BH1, BH2, BHZ
  downsamping from 50 to 10
  removing response using inv for 7D.G30A.BH1
  removing response using inv for 7D.G30A.BH2
  removing response using inv for 7D.G30A.BHZ
  removing response using inv for 7D.G30A.BDH
  downloaded 7D.G30A in 54.727307081222534 seconds.
  saving to 2012_2_2_0_0_0T2012_2_3_0_0_0.h5


## Step 2. Read local data and do correction
We use the wrapper function for tilt and compliance corrections.

In [ ]:
#directory to save the data after TC removal
tcdatadir = '../data/tcremoval'

#tilt and compliance removal parameters
window=7200
overlap=0.2
taper=0.08


In [ ]:
"""
Loop through stations
"""
savetofile=True
if savetofile:
    client=Client('IRIS')
    !rm *.h5
    
correctdictall=dict()
normalizecorrectionplot=True
for ista in stalist:
    t0=time.time()
    """
    a. Downloading data that will be used to compute the transfer functions.
    """
    tr1,tr2,trZ,trP = obs.getdata(net,ista,starttime,endtime,samp_freq=samp_freq,
                                  plot=False,rmresp=rmresp,pre_filt=pre_filt)
    ta=time.time() - t0
    """
    b. Call the wrapper to remove the tilt and compliance noise.
    """
    
    
    correct=obs.TCremoval_wrapper(
        tr1,tr2,trZ,trP,window=window,overlap=overlap,merge_taper=taper,
        qc_freq=[0.004, 0.2],qc_spectra=True,fig_spectra=False,
        save_spectrafig=False,fig_transfunc=False)[2]
    correctdictall[net+"."+ista]=correct
    
    obs.plotcorrection(trZ,correct,normalize=normalizecorrectionplot,freq=[0.005,0.1],
                       size=(12,13),save=True,form='png',xlimit=[49000,55000])
        
    tb=time.time() - t0 - ta
    
    print('all cpu times:')
    print(ta,tb)

    """
    c. Save to ASDF file.
    """
    if savetofile:
        year = trZ.stats.starttime.year
        julday = trZ.stats.starttime.julday
        hour = trZ.stats.starttime.hour
        mnt = trZ.stats.starttime.minute
        sec = trZ.stats.starttime.second
        tstamp = str(year) + '.' + str(julday)+'T'+str(hour)+'-'+str(mnt)+'-'+str(sec)
        fnamebase = trZ.stats.network+'.'+trZ.stats.station+'_'+tstamp+'_LEN'+\
                    str(trZ.stats.endtime-trZ.stats.starttime)+'s'
        tags=[]
        for itr,tr in enumerate([tr1,tr2,trZ,trP],1):
            if len(tr.stats.location) == 0:
                tlocation='00'
            else:
                tlocation=tr.stats.location

            tags.append(tr.stats.channel.lower()+'_'+tlocation.lower())

        fnameraw=fnamebase+'_raw.h5'
        sta_inv=client.get_stations(network=net,station=ista,
                                    starttime=starttime,endtime=endtime,
                                    location='*',level='response')
        print('  saving raw data to '+fnameraw)
        utils.save2asdf(fnameraw,Stream(traces=[tr1,tr2,trZ,trP]),tags,sta_inv=sta_inv)

        fnamecorrect=fnamebase+'_corrected.h5'
        print('  saving corrected data to '+fnamecorrect)
        obs.savecorrection(trZ,correct,fname=fnamecorrect,subset=['ZP-H'],sta_inv=sta_inv,
                           format='asdf',debug=True)